In [1]:
import numpy as np

# quara
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)

In [2]:
# 1-qubit Quantum State Tomography

In [3]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
print(c_sys)

elemental_systems:
[0] 0 (system_id=4382055440)

dim: 2
basis:
(array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]]))


In [4]:
# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)
for i, tester in enumerate(testers):
    print("Tester ", i, ":\n", tester)

Tester  0 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]]
Tester  1 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]]
Tester  2 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [5]:
# Quantum State Tomography
seed = 7896
qst = StandardQst(testers, on_para_eq_constraint=False, schedules="all", seed=seed)

In [6]:
mode = "state"
name = "a"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
State

Dim:
2

Vec:
[0.70710678 0.5        0.5        0.        ]


In [7]:
prob_dists = qst.calc_prob_dists(true)
print(prob_dists)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

In [8]:
#qst.reset_seed()
num_data = 1000
empi_dists = qst.generate_empi_dists(state=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.864, 0.136]))
(1000, array([0.844, 0.156]))
(1000, array([0.49, 0.51]))


In [9]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.51477374  0.48648947 -0.01414214]
is estimate physical? :  False

Eigenvalues are:  [1.0009311329913526, -0.0009311329913525568]


In [10]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError()
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()

#obj_start = (qst.generate_empty_estimation_obj_with_setting_info().generate_origin_obj())
#var_start = obj_start.to_var()
algo_option = ProjectedGradientDescentBacktrackingOption(mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable", num_history_stopping_criterion_gradient_descent=1)

result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710682  0.5138169   0.4855852  -0.01411585]

is estimate physical? :  True

Eigenvalues are:  [1.0000000498185366, 0.0]


In [11]:
# POVM Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names = ["x0", "y0", "z0", "z1"]
testers = generate_tester_states(c_sys=c_sys, names = names)
povmt = StandardPovmt(testers, num_outcomes=2, on_para_eq_constraint=True, schedules="all")

In [12]:
mode = "povm"
name = "z"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


In [13]:
prob_dists = povmt.calc_prob_dists(true)
print(prob_dists)

[[5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [1.00000000e+00 2.22044605e-16]
 [0.00000000e+00 1.00000000e+00]]


In [14]:
num_data = 1000
empi_dists = povmt.generate_empi_dists(povm=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.488, 0.512]))
(1000, array([0.516, 0.484]))
(1000, array([1., 0.]))
(1000, array([0., 1.]))


In [15]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678 -0.01697056  0.02262742  0.70710678]
 [ 0.70710678  0.01697056 -0.02262742 -0.70710678]]

is estimate physical? :  False

Eigenvalues are: [[1.000399840127872, -0.0003998401278722573], [1.000399840127872, -0.0003998401278720354]]


In [16]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(4)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

ValueError: operands could not be broadcast together with shapes (4,) (3,) 

In [17]:
# Quantum Process Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names_states = ["x0", "y0", "z0", "z1"]
testers_states = generate_tester_states(c_sys=c_sys, names = names_states)
names_povms = ["x", "y", "z"]
testers_povms = generate_tester_povms(c_sys=c_sys, names=names_povms)
qpt = StandardQpt(states=testers_states, povms=testers_povms, on_para_eq_constraint=True, schedules="all")

In [19]:
mode = "gate"
name = "hadamard"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]


In [20]:
prob_dists = qpt.calc_prob_dists(true)
print(prob_dists)

[[0.5 0.5]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [0.5 0.5]]


In [21]:
num_data = 1000
empi_dists = qpt.generate_empi_dists(gate=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.507, 0.493]))
(1000, array([0.513, 0.487]))
(1000, array([1., 0.]))
(1000, array([0.47, 0.53]))
(1000, array([0., 1.]))
(1000, array([0.471, 0.529]))
(1000, array([1., 0.]))
(1000, array([0.505, 0.495]))
(1000, array([0.476, 0.524]))
(1000, array([0., 1.]))
(1000, array([0.505, 0.495]))
(1000, array([0.524, 0.476]))


In [22]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.     0.     0.     0.   ]
 [ 0.     0.014 -0.06   1.   ]
 [ 0.01   0.016 -1.01   0.   ]
 [ 0.     1.    -0.058 -0.048]]

is estimate physical? :  False

Eigenvalues are: [-0.05842999  0.01079319  0.04147905  2.00615775]


In [25]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError()
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

ValueError: operands could not be broadcast together with shapes (12,) (3,) 